<a href="https://colab.research.google.com/github/AbimaelLopez/Diplomado/blob/main/Ejercicio_10_febrero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

5. Ejercicio: Árboles de decisión para problema de regresión
Utilizar árboles de decisión para predecir el valor incluido en 'MEDV' del conjunto de datos Boston house.

Leer los datos y separarlos correctamente
Definir un pipeline haciendo uso de ColumnTransformer. Identifica las columnas numéricas y define un imputador con estrategia de media y un escalador Standard.
Importar la clase correspondiente a árboles de decisión para regresión
Hacer los cambios necesarios para trabajar con problemas de regresión, incluyendo no usar estratificación en la separación de datos y revisar las métricas.
Define un espacio de búsqueda de hiperparámetros del árbol. También considera que quizás debas cambiar los criterios del árbol para trabajar con problemas de regresión. Revisa la documentación de la clase.
Haz la búsqueda de hiperparámetros con GridSearchCV. Usa la métrica MSE.
Evalúa en el conjunto de prueba.
http://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html
https://raw.githubusercontent.com/JoaquinAmatRodrigo/Estadistica-machine-learning-python/master/data/Boston.csv


In [ ]:
#Básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Datasets
from sklearn.datasets import load_iris

#Modelos
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree

#Preprocesamiento
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

#Data split y seleccion de modelos
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

#Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

#Metricas
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [ ]:
url="https://raw.githubusercontent.com/JoaquinAmatRodrigo/Estadistica-machine-learning-python/master/data/Boston.csv"
df=pd.read_csv(url)


In [ ]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


In [ ]:
X = df[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'LSTAT']]
y = df['MEDV']

numerical_features = X.select_dtypes(include=['int64', 'float64']).columns


In [ ]:
#Separación train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
# Create preprocessing steps for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])



In [ ]:

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features)
    ])

In [ ]:
#Creación del pipeline
# Define the scaler
#scaler = MinMaxScaler()

# Define the KNN model
tree_model = DecisionTreeRegressor()

# Create a pipeline with scaler and KNN classifier
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', DecisionTreeRegressor())])



In [ ]:
# Define the hyperparameters to search
param_grid = {
    'regressor__criterion': ['squared_error','absolute_error','friedman_mse', 'poisson'],
   # 'regressor__splitter': ['best', 'random'],
    'regressor__max_depth': [None, 5, 10, 15, 20],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [ 1, 2, 4,8]
}

# Define the cross-validation scheme (StratifiedKFold)
#kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform grid search with cross-validation on the pipeline
grid_search = GridSearchCV(pipeline, param_grid, scoring='neg_mean_squared_error', error_score='raise')

# Fit the data using GridSearchCV
grid_search.fit(X_train, y_train)

GridSearchCV(error_score='raise',
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'LSTAT'],
      dtype='object'))])),
                                       ('regressor', DecisionTreeRegressor())]),
             param_grid={'regressor__criterion': ['squared_error',
                                                  'absolute_error',
                                                  'friedman_mse', 'poisson'],
                         'regressor__max_depth': [None, 5, 10, 15, 20],
                         'regressor__min_samples_leaf': [1, 2, 4, 8],
                         'regressor__min_samples_split': [2, 5, 10]},
             scoring='neg_mean_squared_error')

In [ ]:
#Get the best model
best_model = grid_search.best_estimator_

# Fit the best model on the training data
best_model.fit(X_train, y_train)

# Print the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'regressor__criterion': 'poisson', 'regressor__max_depth': 5, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 10}


In [ ]:
# Evalauamos en el train set solo para ver que no quedó sobreajustado
# Evaluate the model on the train set
from sklearn.metrics import mean_squared_error
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print('mse: ', mse)

train_accuracy = best_model.score(X_train, y_train)
print("Train Accuracy:", train_accuracy)

# Evaluate the model on the test set
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy:", test_accuracy)




mse:  25.110943174467383
Train Accuracy: 0.9297990931844173
Test Accuracy: 0.6984232043469034
